<a href="https://colab.research.google.com/github/kaushikabhishek87/Tensorflow_projects/blob/main/11_text_generation_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b5a743ed-ce68-3eb2-0017-35440964a8d6)


In [5]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time
import pandas as pd

In [6]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt

--2021-07-06 06:32:46--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 172.217.9.208, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2021-07-06 06:32:46 (150 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [7]:
text = open("/content/shakespeare.txt", mode="rb" ).read().decode(encoding="utf8")

In [8]:
# text = open("/content/shakespeare.txt", mode="r" ).read()

In [9]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [10]:
# Unique characters 
vocab = set(text)
print(len(vocab))

65


## Preprocess Text

In [11]:
example_text = ["there was just an earthquake", "hoping god everyone is safe"]
chars = tf.strings.unicode_split(example_text, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j', b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't', b'h', b'q', b'u', b'a', b'k', b'e'], [b'h', b'o', b'p', b'i', b'n', b'g', b' ', b'g', b'o', b'd', b' ', b'e', b'v', b'e', b'r', b'y', b'o', b'n', b'e', b' ', b'i', b's', b' ', b's', b'a', b'f', b'e']]>

In [12]:
chars.shape

TensorShape([2, None])

In [13]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab) , 
                                            mask_token=None)

In [14]:
chars[0]

<tf.Tensor: shape=(28,), dtype=string, numpy=
array([b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j',
       b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't',
       b'h', b'q', b'u', b'a', b'k', b'e'], dtype=object)>

In [15]:
ids = ids_from_chars(chars)
ids[0]

<tf.Tensor: shape=(28,), dtype=int64, numpy=
array([22, 55,  1, 51,  1, 19, 43, 31, 57, 19, 39, 24, 57, 22, 19, 31, 35,
       19,  1, 31, 51, 22, 55, 42, 24, 31, 14,  1])>

In [16]:
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(),
                                            invert = True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b't', b'h', b'e', b'r', b'e', b' ', b'w', b'a', b's', b' ', b'j', b'u', b's', b't', b' ', b'a', b'n', b' ', b'e', b'a', b'r', b't', b'h', b'q', b'u', b'a', b'k', b'e'], [b'h', b'o', b'p', b'i', b'n', b'g', b' ', b'g', b'o', b'd', b' ', b'e', b'v', b'e', b'r', b'y', b'o', b'n', b'e', b' ', b'i', b's', b' ', b's', b'a', b'f', b'e']]>

In [17]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'there was just an earthquake', b'hoping god everyone is safe'],
      dtype=object)

In [18]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy()

In [19]:
text_from_ids(ids)

array([b'there was just an earthquake', b'hoping god everyone is safe'],
      dtype=object)

## Creating Training Exmaples & Target

In [20]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, input_encoding="UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([65, 18, 51, ..., 53, 25, 37])>

In [21]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [22]:
for i in ids_dataset.take(10):
  print(chars_from_ids(i).numpy().decode("UTF-8") )


F
i
r
s
t
 
C
i
t
i


In [23]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

11043

In [24]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for i in sequences.take(1):
  print(chars_from_ids(i))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [25]:
for i in sequences.take(5):
  print(text_from_ids(i))

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [26]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


In [27]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [28]:
dataset = sequences.map(split_input_target)

In [29]:
for input_example, target_example in dataset.take(1):
  print(text_from_ids(input_example))
  print(text_from_ids(target_example))


b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Creating training batches

In [30]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)
)
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [31]:
dataset.take(1)

<TakeDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Model

In [32]:
embedding = tf.keras.layers.Embedding(input_dim=len(ids_from_chars.get_vocabulary()), output_dim=256)

inputs = tf.keras.layers.Input(shape=(100,) )
X = embedding(inputs)
X = tf.keras.layers.GRU(units= 1024, return_state=False, return_sequences=True)(X)
outputs = tf.keras.layers.Dense(len(ids_from_chars.get_vocabulary()))(X)

model = tf.keras.models.Model(inputs, outputs)

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 256)          16896     
_________________________________________________________________
gru (GRU)                    (None, 100, 1024)         3938304   
_________________________________________________________________
dense (Dense)                (None, 100, 66)           67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [35]:
model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer="Adam",
              metrics=["accuracy"])

model.fit(dataset, epochs=20)

Epoch 1/20
172/172 [==============================] - 11s 52ms/step - loss: 2.7306 - accuracy: 0.2811
Epoch 2/20
172/172 [==============================] - 9s 51ms/step - loss: 2.0102 - accuracy: 0.4139
Epoch 3/20
172/172 [==============================] - 9s 52ms/step - loss: 1.7414 - accuracy: 0.4847
Epoch 4/20
172/172 [==============================] - 9s 52ms/step - loss: 1.5729 - accuracy: 0.5294
Epoch 5/20
172/172 [==============================] - 9s 53ms/step - loss: 1.4674 - accuracy: 0.5570
Epoch 6/20
172/172 [==============================] - 9s 53ms/step - loss: 1.3945 - accuracy: 0.5751
Epoch 7/20
172/172 [==============================] - 9s 53ms/step - loss: 1.3391 - accuracy: 0.5895
Epoch 8/20
172/172 [==============================] - 9s 54ms/step - loss: 1.2926 - accuracy: 0.6014
Epoch 9/20
172/172 [==============================] - 9s 54ms/step - loss: 1.2509 - accuracy: 0.6119
Epoch 10/20
172/172 [==============================] - 9s 54ms/step - loss: 1.2098 - accur